# Q2

During the 3-month period from 1st Oct to 31st Dec 2023, which shipper(s) should be notified automatically regarding potential late delivery of which shipments, and at what times

# 1. Library Import

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import helper

from datetime import datetime
from datetime import timedelta

# 2. Data Loading

In [2]:
df_GPS = pd.read_csv('Data/GPS_data.csv')
df_bookings = pd.read_csv('Data/Shipment_bookings.csv')
df_new_bookings = pd.read_csv('Data/New_bookings.csv')

print(df_bookings.shape, df_new_bookings.shape, df_GPS.shape)

(3245, 15) (1000, 15) (641662, 4)


In [3]:
df_bookings.FIRST_COLLECTION_POST_CODE.isna().sum()

1

In [4]:
df = pd.read_csv('Data/bookings_processed.csv')

# 3. Late Collection & Late Delivery Notifications

In [5]:
# convert to datetime format 
date_cols = ['FIRST_COLLECTION_SCHEDULE_EARLIEST', 
             'FIRST_COLLECTION_SCHEDULE_LATEST', 
             'LAST_DELIVERY_SCHEDULE_EARLIEST',
             'LAST_DELIVERY_SCHEDULE_LATEST',
             'RECORD_TIMESTAMP']

for i in date_cols:
    df[i]=pd.to_datetime(df[i])

In [6]:
df.head()

,Unnamed: 0,PROJECT_ID,SHIPMENT_NUMBER,CARRIER_DISPLAY_ID,VEHICLE_SIZE,VEHICLE_BUILD_UP,FIRST_COLLECTION_POST_CODE,LAST_DELIVERY_POST_CODE,FIRST_COLLECTION_LATITUDE,FIRST_COLLECTION_LONGITUDE,...,LAST_DELIVERY_SCHEDULE_EARLIEST,LAST_DELIVERY_SCHEDULE_LATEST,LAT,LON,RECORD_TIMESTAMP,GPS_coord,last_delivery_coord,POSTCODE,on_time,almost_on_time
0,0,5456fcfa-5b0c-428e-b322-240d508b6660,SEZHUK-231005-250448,CAUK000096,Tractor Unit,Curtain-Side 13.6M Trailer,PE29 2XR,EH48 2FB,52.315672,-0.157664,...,2023-10-09 09:55:00+00:00,2023-10-09 10:25:00+00:00,52.316544,-0.157263,2023-10-06 10:24:00+01:00,"[52.31654422, -0.15726316]","[55.877803, -3.648558]",PE29 2XJ,0,0
1,1,eb7a9df0-4875-43d8-9060-c79c35eae2fb,SEZHUK-231217-275004,CAUK000660,Tractor Unit,Box 44ft,BD5 8LZ,RG19 4BQ,53.764125,-1.740145,...,2023-12-20 06:00:00+00:00,2023-12-20 08:56:00+00:00,53.745423,-1.702855,2023-12-20 04:08:00+00:00,"[53.7454231574671, -1.70285456004236]","[51.398851, -1.231993]",No postcode found for the given coordinates.,0,0
2,2,eb7a9df0-4875-43d8-9060-c79c35eae2fb,SEZHUK-231210-272018,CAUK000435,Tractor Unit,Box 44ft,BD5 8LZ,EN3 7QP,53.767302,-1.746560,...,2023-12-14 07:03:00+00:00,2023-12-14 10:43:00+00:00,51.659889,-0.024926,2023-12-14 10:18:00+00:00,"[51.659889300233104, -0.024926167738865]","[51.6607, -0.022247]",No postcode found for the given coordinates.,0,0
3,3,40415fe3-e446-4d1e-b141-3698b0dc47f3,SEZHUK-231211-272127,CAUK000096,Tractor Unit,Curtain-Side 13.6M Trailer,HP22 5BL,NG31 7UH,51.779657,-0.772897,...,2023-12-13 15:30:00+00:00,2023-12-13 16:00:00+00:00,52.889862,-0.634389,2023-12-13 17:22:00+00:00,"[52.8898615976303, -0.63438873938486]","[52.888113, -0.634948]",No postcode found for the given coordinates.,0,0
4,4,39a4e234-5d35-4d94-8e8a-b8bf6fe13953,SEZHUK-231025-256168,CAUK000155,Tractor Unit,Curtain Side,CV23 8BQ,G75 0ZZ,52.334853,-1.180450,...,2023-11-07 23:00:00+00:00,2023-11-07 23:59:00+00:00,55.845446,-3.880531,2023-11-08 03:53:00+00:00,"[55.8454462753676, -3.88053080781484]","[55.735089, -4.168719]",No postcode found for the given coordinates.,0,0


## 3.1 Processing

In [7]:
## Shippers should be notified when 1st RECORD_TIMESTAMP > LAST_DELIVERY_SCHEDULE_EARLIEST 

df_all=df_GPS.merge(df, how='left', on='SHIPMENT_NUMBER')
df_all.shape

(641662, 27)

In [8]:
df_all.columns

Index(['SHIPMENT_NUMBER', 'LAT_x', 'LON_x', 'RECORD_TIMESTAMP_x', 'Unnamed: 0',
       'PROJECT_ID', 'CARRIER_DISPLAY_ID', 'VEHICLE_SIZE', 'VEHICLE_BUILD_UP',
       'FIRST_COLLECTION_POST_CODE', 'LAST_DELIVERY_POST_CODE',
       'FIRST_COLLECTION_LATITUDE', 'FIRST_COLLECTION_LONGITUDE',
       'LAST_DELIVERY_LATITUDE', 'LAST_DELIVERY_LONGITUDE',
       'FIRST_COLLECTION_SCHEDULE_EARLIEST',
       'FIRST_COLLECTION_SCHEDULE_LATEST', 'LAST_DELIVERY_SCHEDULE_EARLIEST',
       'LAST_DELIVERY_SCHEDULE_LATEST', 'LAT_y', 'LON_y', 'RECORD_TIMESTAMP_y',
       'GPS_coord', 'last_delivery_coord', 'POSTCODE', 'on_time',
       'almost_on_time'],
      dtype='object')

In [9]:
df_all.drop(columns=['LAT_y', 'LON_y', 'RECORD_TIMESTAMP_y'], axis=1, inplace=True)
# df_all.drop(columns=['expected_duration_longest', 'latest_latest'], axis=1, inplace=True)
df_all = df_all.rename(columns={'LAT_x':'LAT', 'LON_x':'LON', 'RECORD_TIMESTAMP_x':'RECORD_TIMESTAMP'})
df_all.columns

Index(['SHIPMENT_NUMBER', 'LAT', 'LON', 'RECORD_TIMESTAMP', 'Unnamed: 0',
       'PROJECT_ID', 'CARRIER_DISPLAY_ID', 'VEHICLE_SIZE', 'VEHICLE_BUILD_UP',
       'FIRST_COLLECTION_POST_CODE', 'LAST_DELIVERY_POST_CODE',
       'FIRST_COLLECTION_LATITUDE', 'FIRST_COLLECTION_LONGITUDE',
       'LAST_DELIVERY_LATITUDE', 'LAST_DELIVERY_LONGITUDE',
       'FIRST_COLLECTION_SCHEDULE_EARLIEST',
       'FIRST_COLLECTION_SCHEDULE_LATEST', 'LAST_DELIVERY_SCHEDULE_EARLIEST',
       'LAST_DELIVERY_SCHEDULE_LATEST', 'GPS_coord', 'last_delivery_coord',
       'POSTCODE', 'on_time', 'almost_on_time'],
      dtype='object')

In [10]:
df_all.shape

(641662, 24)

## 3.2 Late Collection

In [11]:
# number of missing tracking postcodes
df_all['POSTCODE'].isna().sum()

528

### 3.2.1 Impute missing Tracking POSTCODEs

In [12]:
# find missing tracking POST_CODEs & save them in a temporary dataset
df_missing = df_all[df_all['POSTCODE'].isna()]

# combine lon & lat columns into 1 GPS coordinates column
df_missing['GPS_coord']=df_missing[['LAT', 'LON']].values.tolist()

# convert GPS to postcode
df_missing['POSTCODE'] = df_missing['GPS_coord'].apply(helper.get_postcode_from_gps)

df_all['POSTCODE'] = df_all['POSTCODE'].fillna(df_missing['POSTCODE'])
df_all['POSTCODE'].isna().sum()

C:\Users\Lenovo IdeaCentre\AppData\Local\Temp\ipykernel_4592\1880291068.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing['GPS_coord']=df_missing[['LAT', 'LON']].values.tolist()
C:\Users\Lenovo IdeaCentre\AppData\Local\Temp\ipykernel_4592\1880291068.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing['POSTCODE'] = df_missing['GPS_coord'].apply(helper.get_postcode_from_gps)


0

### 3.2.2 Impute missing FIRST_COLLECTION_POST_CODEs

In [13]:
# number of missing FIRST_COLLECTION_POST_CODEs
df_all[df_all.FIRST_COLLECTION_POST_CODE.isna()].shape

(877, 24)

In [14]:
# find missing FIRST_COLLECTION_POST_CODEs & save them in a temporary dataset
df_missing = df_all[df_all['FIRST_COLLECTION_POST_CODE'].isna()]

# combine lon & lat columns into 1 GPS coordinates column
df_missing['FIRST_COLLECTION_coord']=df_missing[['FIRST_COLLECTION_LATITUDE', 'FIRST_COLLECTION_LONGITUDE']].values.tolist()

# convert GPS to postcode
df_missing['FIRST_COLLECTION_POST_CODE'] = df_missing['FIRST_COLLECTION_coord'].apply(helper.get_postcode_from_gps)

df_all['FIRST_COLLECTION_POST_CODE'] = df_all['FIRST_COLLECTION_POST_CODE'].fillna(df_missing['FIRST_COLLECTION_POST_CODE'])
df_all['FIRST_COLLECTION_POST_CODE'].isna().sum()

C:\Users\Lenovo IdeaCentre\AppData\Local\Temp\ipykernel_4592\494028282.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing['FIRST_COLLECTION_coord']=df_missing[['FIRST_COLLECTION_LATITUDE', 'FIRST_COLLECTION_LONGITUDE']].values.tolist()
C:\Users\Lenovo IdeaCentre\AppData\Local\Temp\ipykernel_4592\494028282.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing['FIRST_COLLECTION_POST_CODE'] = df_missing['FIRST_COLLECTION_coord'].apply(helper.get_postcode_from_gps)


0

In [15]:
# label shipments that reached collection postcodes after the latest sheduled collection
df_all['notify_collection'] = np.where((df_all['RECORD_TIMESTAMP']>df_all['FIRST_COLLECTION_SCHEDULE_LATEST'])&
                                      (df_all['POSTCODE'].str[:-1]==df_all['FIRST_COLLECTION_POST_CODE'].str[:-1]), 1, 0)
# keep earliest timestamp for each shipment & store in a temporary dataset 
df_notify_collection = df_all[df_all['notify_collection']==1].sort_values(by='RECORD_TIMESTAMP').drop_duplicates(subset='SHIPMENT_NUMBER', keep='first')
print('Number of shipments to be notified of potential late collection: ', df_notify_collection.shape[0])

Number of shipments to be notified of potential late collection:  20


## 3.3 Late Delivery

In [16]:
# label shipments that reached delivery postcodes after the latest scheduled delivery
df_all['notify_delivery'] = np.where((df_all['RECORD_TIMESTAMP']>df_all['LAST_DELIVERY_SCHEDULE_EARLIEST'])&
                                     (df_all['POSTCODE'].str[:-1]==df_all['LAST_DELIVERY_POST_CODE'].str[:-1]), 1, 0)

# keep earliest timestamp for each shipment & store in a temporary dataset 
df_notify_delivery = df_all[df_all['notify_delivery']==1].sort_values(by='RECORD_TIMESTAMP').drop_duplicates(subset='SHIPMENT_NUMBER', keep='first')

print('Number of shipments to be notified of potential late delivery: ', df_notify_delivery.shape[0])
print('Number of shipments notified of potential late delivery but arrived on time: ', df_notify_delivery[df_notify_delivery['almost_on_time']==1].shape[0])

Number of shipments to be notified of potential late delivery:  317
Number of shipments notified of potential late delivery but arrived on time:  176


## 3.4 Clean & Consolidate Datasets

In [17]:
# only keep necessarry columns for merging with booking dataset
collect=pd.DataFrame(df_notify_collection[['SHIPMENT_NUMBER','notify_collection', 'RECORD_TIMESTAMP']])
collect.columns = ['SHIPMENT_NUMBER','notify_late_collection', 'when_notify_late_collection']
collect.head()

,SHIPMENT_NUMBER,notify_late_collection,when_notify_late_collection
92767,SEZHUK-231014-253149,1,2023-10-15T16:41:00.000+0100
124039,SEZHUK-231019-254595,1,2023-10-22T21:28:00.000+0100
131404,SEZHUK-231020-255020,1,2023-10-23T16:37:00.000+0100
143886,SEZHUK-231024-255918,1,2023-10-24T20:22:00.000+0100
153125,SEZHUK-231025-256450,1,2023-11-05T02:48:00.000Z


In [18]:
# only keep necessarry columns for merging with booking dataset
deliver=pd.DataFrame(df_notify_delivery[['SHIPMENT_NUMBER','notify_delivery', 'RECORD_TIMESTAMP']])
deliver.columns = ['SHIPMENT_NUMBER','notify_late_delivery', 'when_notify_late_delivery']
deliver.head()

,SHIPMENT_NUMBER,notify_late_delivery,when_notify_late_delivery
11842,SEZHUK-230930-249069,1,2023-10-02T08:55:00.000+0100
11468,SEZHUK-230930-249060,1,2023-10-02T08:55:00.000+0100
22209,SEZHUK-231002-249324,1,2023-10-02T21:31:00.000+0100
7273,SEZHUK-230929-248494,1,2023-10-03T08:34:00.000+0100
19798,SEZHUK-231001-249231,1,2023-10-03T08:55:00.000+0100


In [19]:
# join late collection notification dataset with shipment bookings dataset
df=df_bookings.set_index('SHIPMENT_NUMBER').join(collect.set_index('SHIPMENT_NUMBER'))
df.shape

(3245, 16)

In [20]:
# join late delivery notification dataset with shipment bookings dataset
df=df.reset_index()
df=df.set_index('SHIPMENT_NUMBER').join(deliver.set_index('SHIPMENT_NUMBER'))
df=df.reset_index()
df.shape

(3245, 19)

In [21]:
# impute shipments that don't need to be notified of possible delay with 0

df['notify_late_collection']=df['notify_late_collection'].fillna(0) 
df['notify_late_delivery']=df['notify_late_delivery'].fillna(0)

In [22]:
# The final dataset contains a flag that labels whether or not a notification needs to be sent to a shipper 
# to warn them of potential late collection and late delivery 
# and a timestamp when each notification needs to be sent respectively

# The following is a shipment that should've been notified of both late collection & delivery

df[(df['notify_late_collection']==1)&(df['notify_late_delivery']==1)][['SHIPMENT_NUMBER', 
                                                                       'notify_late_collection', 
                                                                       'when_notify_late_collection',
                                                                       'notify_late_delivery', 
                                                                       'when_notify_late_delivery']]

,SHIPMENT_NUMBER,notify_late_collection,when_notify_late_collection,notify_late_delivery,when_notify_late_delivery
926,SEZHUK-231216-274798,1.0,2023-12-21T04:00:00.000Z,1.0,2023-12-21T04:00:00.000Z


# 4. Save Datasets

In [23]:
# df_notify_collection.to_csv('Data/late_collection_notification.csv')
# df_notify_delivery.to_csv('Data/late_delivery_notification.csv')
df.to_csv('Data/Shipment_bookings_with_notifications.csv')

In [ ]:
# df_notify_delivery = pd.read_csv('Data/late_delivery_notification.csv')
# df_notify_collection = pd.read_csv('Data/late_collection_notification.csv')